In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split

# load data
data = pd.read_csv('./data/melb_data.csv')

y = data.Price

melb_predictors = data.drop('Price', axis=1)
X = melb_predictors.select_dtypes(exclude=['object'])
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# print(score_dataset(reduce_X_train, reduce_X_valid, y_train, y_valid))

In [10]:
# A Simple Option: Drop Columns with Missing Values
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]
print(f"Missing values: {cols_with_missing}")

reduce_X_train = X_train.drop(cols_with_missing, axis=1)
reduce_X_valid = X_valid.drop(cols_with_missing, axis=1)

print(f"MAE from approach 1 (Drop columns with missing value):")
print(score_dataset(reduce_X_train, reduce_X_valid, y_train, y_valid))

Missing values: ['Car', 'BuildingArea', 'YearBuilt']
MAE from approach 1 (Drop columns with missing value):
183550.22137772635


In [18]:
# A Better Option: Imputation
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train), columns=X_train.columns)
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid), columns=X_valid.columns)

print(f"MAE from approach 2 (Impute columns with missing value):")
print(score_dataset(imputed_X_train, imputed_X_valid, y_train, y_valid))

MAE from approach 2 (Impute columns with missing value):
178166.46269899711


In [19]:
# approach 3 Extension to Imputation

X_train_plus = X_train.copy()
X_valid_plus = X_valid.copy()

for col in cols_with_missing:
    X_train_plus[col + '_was_missing'] = X_train[col].isnull()
    X_valid_plus[col + '_was_missing'] = X_train[col].isnull()
my_imputer = SimpleImputer()
imputed_X_train_plus = pd.DataFrame(my_imputer.fit_transform(X_train_plus), columns=X_train_plus.columns)
imputed_X_valid_plus = pd.DataFrame(my_imputer.transform(X_valid_plus), columns=X_valid_plus.columns)

print("MAE from approach 3 (Impute columns with missing value):")
print(score_dataset(imputed_X_train_plus, imputed_X_valid_plus, y_train, y_valid))

MAE from approach 3 (Impute columns with missing value):
179808.1875113963


In [26]:
# Shape of training data (num_rows, num_columns)
print(X_train.shape)
# print(X_train.isnull().sum())
# Number of missing values in each column of training data
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

(10864, 12)
Car               49
BuildingArea    5156
YearBuilt       4307
dtype: int64
